In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#distance calculations
from geopy.distance import great_circle
from geopy.distance import vincenty

# dictionary building
from collections import defaultdict

#datatime parsing
from dateutil import parser
from datetime import datetime
import time

%matplotlib inline

In [3]:
!ls ~/Documents/UCBerkeleyMIDS/W209/rodents-nyc/data
!cd ~/Documents/UCBerkeleyMIDS/W209/rodents-nyc/data/
!pwd

311_2014_first2000.csv
DOITT_SUBWAY_STATION_01_13SEPT2010.csv
Rodent_Inspection.csv
activate_rat_by_boro_date.csv
active_rat_by_zip.csv
active_rat_seasonal.csv
active_rodent_inspections.csv
all_mta.csv
mta_sightings_only.csv
rest_food_type_avg_violation.csv
restaurant_and_avg_rat_sightings.csv
restaurant_and_avg_rat_sightings_year.csv
rodent_restaurant_ratings.csv
stations_lines_colors.csv
subway_stations_with_zip.csv
subway_stations_with_zip.tsv
subway_stations_with_zip_tab.tsv
test_trimmed_311_2014.csv
trimmed_mta.csv
trimmed_mta_only.csv
year_over_year_diff_by_zip.csv
/Users/ChrisD/Documents/UCBerkeleyMIDS/W209/rodents-nyc


In [4]:
active_rodent_inspections=pd.read_csv("data/active_rodent_inspections.csv",header=0)

In [5]:
test_active_rodent_inspections=active_rodent_inspections.head(20)

In [6]:
test_active_rodent_inspections.columns

Index(['INSPECTION_TYPE', 'JOB_TICKET_OR_WORK_ORDER_ID', 'JOB_ID',
       'JOB_PROGRESS', 'BBL', 'BORO_CODE', 'BLOCK', 'LOT', 'HOUSE_NUMBER',
       'STREET_NAME', 'ZIP_CODE', 'X_COORD', 'Y_COORD', 'LATITUDE',
       'LONGITUDE', 'BOROUGH', 'INSPECTION_DATE', 'RESULT', 'APPROVED_DATE',
       'LOCATION', 'n_prev_visits', 'new_visit'],
      dtype='object')

In [7]:
test_active_rodent_inspections.head(2)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit
0,COMPLIANCE,20090,PO14874,2,3001480046,3,148,46,81,WILLOUGHBY STREET,...,191458.0,40.692184,-73.985753,Brooklyn,2010-05-14 13:46:39,Active Rat Signs,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True
1,COMPLIANCE,24538,PO17193,2,4123940091,4,12394,91,172-04,LINDEN BOULEVARD,...,191395.0,40.691804,-73.777942,Queens,2010-03-03 09:21:09,Active Rat Signs,2010-03-05 11:30:16,"(40.6918037759981, -73.7779418602477)",0,False


### Create Subway station data and dictionary

In [8]:
subway_stations=pd.read_csv('~/Documents/UCBerkeleyMIDS/W209/rodents-nyc/data/subway_stations_with_zip.csv',header=0,dtype={'zip':str,'line':str})

#drop unneeded cols
subway_stations=subway_stations.drop(['URL','NOTES'],axis=1)
subway_stations.head(2)

#change col name
subway_stations.columns=['STATIONID', 'NAME', 'GPS', 'LINE', 'ZIP']

#format text of GPS
subway_stations['GPS']=subway_stations['GPS'].map(lambda x: x.lstrip('POINT ').replace(' ',','))
subway_stations['GPS']=subway_stations['GPS'].map(lambda x: eval(x))

In [9]:
subway_stations.head(2)

,STATIONID,NAME,GPS,LINE,ZIP
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013


In [20]:
#Create dictionary of every GPS point associated with a station (line agnostic)

from collections import defaultdict
subway_dict=defaultdict(list)

for index, row in subway_stations.iterrows():
    #use string of STATION ID for consistency
    subway_dict[str(row['STATIONID'])]={'NAME':row['NAME'],'GPS':row['GPS']}

In [23]:
subway_dict['43']

{'GPS': (-73.89306639507903, 40.823976841237396), 'NAME': 'Simpson St'}

In [30]:
for key,value in subway_dict.items():
    if 'Canal' in subway_dict[key]['NAME']:
        print(subway_dict[key]['NAME'],':',subway_dict[key]['GPS'])

Canal St : (-74.00019299927328, 40.71880300107709)
Canal St : (-74.0062770001748, 40.72285399778783)
Canal St - Holland Tunnel : (-74.0052290023424, 40.72082400007119)
Canal St : (-73.99982638545937, 40.71817387697391)
Canal St : (-74.0018260000577, 40.71946500105898)
Canal St : (-74.00105471306033, 40.718814263587134)


In [75]:

newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(great_circle(newport_ri, cleveland_oh).meters-vincenty(newport_ri, cleveland_oh).meters)

-2240.9385617375374


In [35]:
print(type(subway_stations['GPS'][0]))
subway_stations['GPS'][0]

<class 'tuple'>


(-73.99106999861966, 40.73005400028978)

## DISTANCE TESTING

In [37]:
test_active_rodent_inspections.head(2)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit
0,COMPLIANCE,20090,PO14874,2,3001480046,3,148,46,81,WILLOUGHBY STREET,...,191458.0,40.692184,-73.985753,Brooklyn,2010-05-14 13:46:39,Active Rat Signs,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True
1,COMPLIANCE,24538,PO17193,2,4123940091,4,12394,91,172-04,LINDEN BOULEVARD,...,191395.0,40.691804,-73.777942,Queens,2010-03-03 09:21:09,Active Rat Signs,2010-03-05 11:30:16,"(40.6918037759981, -73.7779418602477)",0,False


In [44]:
for index, row in test_active_rodent_inspections.iterrows():
    #test_point=(row["LONGITUDE"],row["LATITUDE"])
    test_point=row['LOCATION']
    
    for key in subway_dict.keys():
        points=subway_dict[key]['GPS']
        
        for p in [points]:
            #long_mta=p[0]  #station longitude point to test
            #lat_mta=p[1] # station latitude point to test
            #print(p[0])
            if vincenty(test_point, (p[1],p[0])).meters <=100:
                print("index:",index,'Station ID:',key,p,subway_dict[key]['NAME'])

index: 0 Station ID: 377 (-73.98605667854612, 40.69225539645323) Jay St - MetroTech
index: 14 Station ID: 377 (-73.98605667854612, 40.69225539645323) Jay St - MetroTech


In [228]:
for key in subway_dict.keys():
        points=subway_dict[key]['GPS']
        
        for p in [points]:
            #long_mta=p[0]  #station longitude point to test
            #lat_mta=p[1] # station latitude point to test
            #print(p[0])
            if vincenty(test_point, (p[1],p[0])).meters <=100:
                print("index:",index,'Station ID:',key,p,subway_dict[key]['NAME'])

In [55]:
def MTA_distance(subway_dict,test_point,distance):
    
    stations=list()
    
    for key in subway_dict.keys():
        p=subway_dict[key]['GPS']  # gps point associated with MTA station ID
        
        
        if vincenty(test_point, (p[1],p[0])).meters <=distance:   # note that dict() in (long,lat) so easier to reverse it
                stations.append(key)  
                
    if len(stations)>0:
        return stations
    else:
        return '_'

#### Map on Test Set

In [54]:
print('unique station id:',len(subway_stations.STATIONID.unique()))
print('unique GPS:',len(subway_stations.GPS.unique()))
print('unique NAME:',len(subway_stations.NAME.unique()))

unique station id: 473
unique GPS: 473
unique NAME: 355


In [47]:
test_active_rodent_inspections.head(2)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit
0,COMPLIANCE,20090,PO14874,2,3001480046,3,148,46,81,WILLOUGHBY STREET,...,191458.0,40.692184,-73.985753,Brooklyn,2010-05-14 13:46:39,Active Rat Signs,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True
1,COMPLIANCE,24538,PO17193,2,4123940091,4,12394,91,172-04,LINDEN BOULEVARD,...,191395.0,40.691804,-73.777942,Queens,2010-03-03 09:21:09,Active Rat Signs,2010-03-05 11:30:16,"(40.6918037759981, -73.7779418602477)",0,False


In [56]:
test_active_rodent_inspections["MTA_ID"]=test_active_rodent_inspections['LOCATION'].map(lambda x: MTA_distance(subway_dict,x,100))

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
test_active_rodent_inspections[test_active_rodent_inspections['MTA_ID'] != '_']

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit,MTA_ID
0,COMPLIANCE,20090,PO14874,2,3001480046,3,148,46,81,WILLOUGHBY STREET,...,40.692184,-73.985753,Brooklyn,2010-05-14 13:46:39,Active Rat Signs,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True,[377]
14,COMPLIANCE,28921,PO11761,2,3001520118,3,152,118,141,LAWRENCE STREET,...,40.691739,-73.986283,Brooklyn,2010-05-14 12:15:33,Active Rat Signs,2010-05-18 13:00:33,"(40.6917392459776, -73.9862827561782)",2,True,[377]


#### Test on 1500 for time

In [61]:
test_1500=active_rodent_inspections.head(1500)

In [67]:
start=time.time()
test_1500["MTA_ID"]=test_1500['LOCATION'].map(lambda x: MTA_distance(subway_dict,x,100))
print(time.time()-start)

38.28602695465088


/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [70]:
test_1500.tail(2)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit,MTA_ID
1498,COMPLIANCE,61698,PO57904,2,1005280035,1,528,35,40,DOWNING STREET,...,40.729203,-74.004092,Manhattan,2010-03-03 09:58:41,Active Rat Signs,2010-03-08 13:36:16,"(40.7292032547709, -74.0040915088407)",1,True,_
1499,COMPLIANCE,61715,PO54658,2,5028940001,5,2894,1,231,STEUBEN STREET,...,40.608398,-74.082330,Staten Island,2010-03-04 14:50:00,Active Rat Signs,2010-03-17 14:23:50,"(40.6083981897965, -74.0823303833877)",2,True,_


#### Full dataset

In [71]:
all_mta_ID=active_rodent_inspections

In [72]:
start=time.time()
all_mta_ID['MTA']=all_mta_ID['LOCATION'].map(lambda x: MTA_distance(subway_dict,x,100))
print(time.time()-start)

all_mta_ID.to_csv('all_mta_ID.csv')
all_mta_ID.to_csv('data/all_mta_ID.csv')

4032.138792991638


**took 4023 seconds overnight on local laptop**

In [75]:
all_mta_ID.head(2)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit,MTA
0,COMPLIANCE,20090,PO14874,2,3001480046,3,148,46,81,WILLOUGHBY STREET,...,40.692184,-73.985753,Brooklyn,2010-05-14 13:46:39,Active Rat Signs,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True,[377]
1,COMPLIANCE,24538,PO17193,2,4123940091,4,12394,91,172-04,LINDEN BOULEVARD,...,40.691804,-73.777942,Queens,2010-03-03 09:21:09,Active Rat Signs,2010-03-05 11:30:16,"(40.6918037759981, -73.7779418602477)",0,False,_


In [74]:
mta_ID_only=all_mta_ID[all_mta_ID['MTA']!='_'] #drop rows not 100m from subway station

mta_ID_only['MTA']=mta_ID_only['MTA'].map(lambda x: list(set(x))) #remove duplicates in list

mta_ID_only.to_csv('data/mta_ID_only.csv')
mta_ID_only.shape

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(10123, 23)

In [78]:
mta_ID_only[mta_ID_only['MTA'].map(lambda x:len(x)>1)].head(20)

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,...,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION,n_prev_visits,new_visit,MTA
121,COMPLIANCE,38481,PO16845,2,1007910032,1,791,32,113,WEST 15 STREET,...,40.738341,-73.997012,Manhattan,2010-01-23 08:00:42,Active Rat Signs,2010-01-25 15:14:36,"(40.738341168731, -73.9970118831005)",0,False,"[441, 442]"
394,COMPLIANCE,42713,PO39952,2,4012840069,4,1284,69,37-66,74 STREET,...,40.747183,-73.891508,Queens,2010-01-27 11:20:35,Active Rat Signs,2010-02-02 15:01:22,"(40.74718308167, -73.8915079516895)",0,False,"[231, 438]"
463,COMPLIANCE,42970,PO23875,2,2024840009,2,2484,9,880,RIVER AVENUE,...,40.828333,-73.925625,Bronx,2010-02-01 14:20:56,Active Rat Signs,2010-02-02 16:18:54,"(40.8283329196285, -73.9256252192551)",0,False,"[297, 48]"
1096,COMPLIANCE,54587,PO50634,1,1000530039,1,53,39,96,GREENWICH STREET,...,40.708142,-74.013526,Manhattan,2010-02-04 18:56:35,Active Rat Signs,2010-02-05 11:31:35,"(40.7081419924649, -74.0135258469897)",0,False,"[422, 425]"
1105,COMPLIANCE,54957,PO46945,2,1000190024,1,19,24,50,TRINITY PLACE,...,40.707632,-74.013068,Manhattan,2010-02-02 17:00:24,Active Rat Signs,2010-02-03 10:46:12,"(40.7076315193686, -74.0130676715636)",1,True,"[422, 425]"
1107,COMPLIANCE,54961,PO46982,2,1000530037,1,53,37,102,GREENWICH STREET,...,40.708320,-74.013450,Manhattan,2010-02-04 18:34:54,Active Rat Signs,2010-02-05 11:31:25,"(40.7083204109591, -74.0134501382461)",1,True,"[422, 425]"
1129,COMPLIANCE,55034,PO46926,2,1000490001,1,49,1,75,BROADWAY,...,40.707500,-74.011917,Manhattan,2010-02-02 17:20:26,Active Rat Signs,2010-02-03 10:47:28,"(40.7074998955829, -74.0119170581195)",1,True,"[422, 420]"
1161,COMPLIANCE,55756,PO50142,2,1001330019,1,133,19,57,WARREN STREET,...,40.714757,-74.009281,Manhattan,2010-02-05 18:24:21,Active Rat Signs,2010-02-08 11:46:36,"(40.7147572899728, -74.0092814555548)",1,True,"[404, 361]"
1234,COMPLIANCE,56978,PO50075,2,1001940036,1,194,36,413,BROADWAY,...,40.719078,-74.002168,Manhattan,2010-03-09 10:44:42,Active Rat Signs,2010-03-11 14:08:16,"(40.7190778963362, -74.0021680980926)",1,True,"[435, 417]"
1337,COMPLIANCE,59924,PO55594,2,1002080019,1,208,19,239,CANAL STREET,...,40.718087,-74.000036,Manhattan,2010-02-24 09:34:40,Active Rat Signs,2010-03-08 13:39:51,"(40.7180870585152, -74.0000360743072)",1,True,"[415, 2]"


In [79]:
print(mta_ID_only.shape[0])
print(mta_ID_only[mta_ID_only['MTA'].map(lambda x:len(x)>1)].shape[0])
427/10123

10123
427


0.04218117158944977

Of the 10123 MTA station-adjacent sightings, 427 could be double-counted because they are between two close stations, about 4.2% of the sightings. We will count them as belonging to either station because the audience will be looking at the data on a per-station basis, not as a statistically exact represenation of rat sighting probabilities. 

#### Getting Lines and Colors

In [80]:
subway_stations.head(2)

,STATIONID,NAME,GPS,LINE,ZIP
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013


In [81]:
subway_stations_lines_colors=subway_stations

In [82]:
subway_stations_lines_colors['LINES']=subway_stations_lines_colors['LINE'].map(lambda x: x.split('-'))

In [ ]:

Primary Trunk line	Color[12][13]	Pantone[14]	Hexadecimal	Service bullets
IND Eighth Avenue Line	Vivid blue	PMS 286	#2850ad	"A" train ​"C" train ​"E" train
IND Sixth Avenue Line	Bright orange	PMS 165	#ff6319	"B" train ​"D" train ​"F" train ​"M" train
IND Crosstown Line	Lime green	PMS 376	#6cbe45	"G" train
BMT Canarsie Line	Light slate gray	50% black	#a7a9ac	"L" train
BMT Nassau Street Line	Terra cotta brown	PMS 154	#996633	"J" train ​"Z" train
BMT Broadway Line	Sunflower yellow	PMS 116	#fccc0a	"N" train ​"Q" train ​"R" train ​"W" train
IRT Broadway–Seventh Avenue Line	Tomato red	PMS 185	#ee352e	"1" train ​"2" train ​"3" train
IRT Lexington Avenue Line	Apple green	PMS 355	#00933c	"4" train ​"5" train ​"6" train "6" express train
IRT Flushing Line	Raspberry	PMS Purple	#b933ad	"7" train "7" express train​
Shuttles	Dark slate gray	70% black	#808183	shuttle train

In [83]:
subway_colors={'blue':['A','C','E'],'orange':['B','D','F','M'],'lime':['G'], \
              'grey':['L'],'brown':['J','Z'],'yellow':['N','Q','R','W'], \
              'red':['1','2','3'],'green':['4','5','6','6 Express'],'purple':['7','7 Express'],\
              'silver':['S']}

In [84]:
subway_line_to_color=dict()

for key,value in subway_colors.items():
    for v in value:
        subway_line_to_color[v]=key

In [85]:
lines=list()

for index,row in subway_stations_lines_colors.iterrows():
    
    for L in row['LINES']:
        if L in lines:
            pass
        else:
            lines.append(L)

print(lines)

['4', '6', '6 Express', '1', '2', '3', 'A', 'B', 'C', 'E', 'M', 'R', 'J', 'Z', 'D', 'N', 'Q', 'S', '5', '7', '7 Express', 'F', 'G', 'L', 'W']


In [86]:
subway_stations_lines_colors['COLOR']=None
subway_stations_lines_colors.head(2)

,STATIONID,NAME,GPS,LINE,ZIP,LINES,COLOR
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003,"[4, 6, 6 Express]",None
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013,"[4, 6, 6 Express]",None


In [87]:
# Make colors column

for index,row in subway_stations_lines_colors.iterrows():
    
    colors=list()
    
    for L in row['LINES']:
        c=subway_line_to_color[L]  #lookup color of subway line
        
        if c not in colors:
            #print('hi')
            colors.append(c)
    
    subway_stations_lines_colors['COLOR'].iloc[index]=colors
    #print(colors, row['COLOR'])

subway_stations_lines_colors.head(2)

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,STATIONID,NAME,GPS,LINE,ZIP,LINES,COLOR
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003,"[4, 6, 6 Express]",[green]
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013,"[4, 6, 6 Express]",[green]


In [88]:
subway_stations_lines_colors.head(20)

,STATIONID,NAME,GPS,LINE,ZIP,LINES,COLOR
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003,"[4, 6, 6 Express]",[green]
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013,"[4, 6, 6 Express]",[green]
2,3,50th St,"(-73.98384899986625, 40.76172799961419)",1-2,10019,"[1, 2]",[red]
3,4,Bergen St,"(-73.97499915116808, 40.68086213682956)",2-3-4,11217,"[2, 3, 4]","[red, green]"
4,5,Pennsylvania Ave,"(-73.89488591154061, 40.66471445143568)",3-4,11207,"[3, 4]","[red, green]"
5,6,238th St,"(-73.90087000018522, 40.88466700064975)",1,10463,[1],[red]
6,7,Cathedral Pkwy (110th St),"(-73.95806670661364, 40.800581558114956)",A-B-C,10036,"[A, B, C]","[blue, orange]"
7,8,Kingston - Throop Aves,"(-73.94085899871263, 40.67991899941601)",A-C,11216,"[A, C]",[blue]
8,9,65th St,"(-73.8987883783301, 40.74971952935675)",E-M-R,11377,"[E, M, R]","[blue, orange, yellow]"
9,10,36th St,"(-73.92901818461539, 40.75196004401078)",E-M-R,11101,"[E, M, R]","[blue, orange, yellow]"


In [89]:
subway_stations_lines_colors.to_csv('data/stations_lines_colors.csv')
!head data/stations_lines_colors.csv

,STATIONID,NAME,GPS,LINE,ZIP,LINES,COLOR
0,1,Astor Pl,"(-73.99106999861966, 40.73005400028978)",4-6-6 Express,10003,"['4', '6', '6 Express']",['green']
1,2,Canal St,"(-74.00019299927328, 40.71880300107709)",4-6-6 Express,10013,"['4', '6', '6 Express']",['green']
2,3,50th St,"(-73.98384899986625, 40.76172799961419)",1-2,10019,"['1', '2']",['red']
3,4,Bergen St,"(-73.97499915116808, 40.68086213682956)",2-3-4,11217,"['2', '3', '4']","['red', 'green']"
4,5,Pennsylvania Ave,"(-73.89488591154061, 40.66471445143568)",3-4,11207,"['3', '4']","['red', 'green']"
5,6,238th St,"(-73.90087000018522, 40.88466700064975)",1,10463,['1'],['red']
6,7,Cathedral Pkwy (110th St),"(-73.95806670661364, 40.800581558114956)",A-B-C,10036,"['A', 'B', 'C']","['blue', 'orange']"
7,8,Kingston - Throop Aves,"(-73.94085899871263, 40.67991899941601)",A-C,11216,"['A', 'C']",['blue']
8,9,65th St,"(-73.8987883783301, 40.74971952935675)",E-M-R,11377,"['E', 'M', 'R']","['blue', 'orange', 'yellow']"


In [94]:
subway_stations_lines_colors

subway_stations_lines_colors.index[subway_stations_lines_colors['NAME']=='103rd St']

Int64Index([158, 160, 457], dtype='int64')

### Add lines and colors

In [96]:
print('total',mta_ID_only.shape[0])

print('JOB_ID',len(mta_ID_only['JOB_ID'].unique()))
print('JOB_TICKET_OR_WORK_ORDER_ID',len(mta_ID_only['JOB_TICKET_OR_WORK_ORDER_ID'].unique()))
print('BBL',len(mta_ID_only['BBL'].unique()))


total 10123
JOB_ID 7040
JOB_TICKET_OR_WORK_ORDER_ID 10123
BBL 3256


In [101]:
for i in mta_ID_only.columns:
    print(i, len(mta_ID_only[i].unique()) )

INSPECTION_TYPE 2
JOB_TICKET_OR_WORK_ORDER_ID 10123
JOB_ID 7040
JOB_PROGRESS 21
BBL 3256
BORO_CODE 4
BLOCK 976
LOT 195
HOUSE_NUMBER 1888
STREET_NAME 851
ZIP_CODE 115
X_COORD 4224
Y_COORD 4333
LATITUDE 4972
LONGITUDE 5091
BOROUGH 4
INSPECTION_DATE 10122
RESULT 1
APPROVED_DATE 8652
LOCATION 5115
n_prev_visits 30
new_visit 2


TypeError: unhashable type: 'list'

In [102]:
newcols= ['JOB_TICKET_OR_WORK_ORDER_ID','BORO_CODE', 'BOROUGH', 'ZIP_CODE', 'LATITUDE',
       'LONGITUDE', 'INSPECTION_DATE', 'APPROVED_DATE',
       'LOCATION', 'n_prev_visits', 'new_visit', 'MTA']

In [103]:
trimmed_mta_ID_only=mta_ID_only[newcols]

In [104]:
trimmed_mta_ID_only.to_csv('data/trimmed_mta.csv')

In [455]:
trimmed_mta_ID_only['COLOR']='-'
trimmed_mta_ID_only['LINES']='-'

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [119]:
subway_stations_lines_colors.index[subway_stations_lines_colors['STATIONID']==3][0]

2

In [125]:
type(trimmed_mta_ID_only['MTA'].iloc[3][0])

str

In [131]:
subway_stations_lines_colors['STATIONID']==3

0      False
1      False
2       True
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
443    False
444    False
445    False
446    False
447    False
448    False
449    False
450    False
451    False
452    False
453    False
454    False
455    False
456    False
457    False
458    False
459    False
460    False
461    False
462    False
463    False
464    False
465    False
466    False
467    False
468    False
469    False
470    False
471    False
472    False
Name: STATIONID, Length: 473, dtype: bool

In [132]:
def get_mta_color(ID,subway_df):
    
    color=list() #initiate colors
    
    for idee in ID:
        
        #find index of subway station
        stat_index=subway_df.index[subway_df['STATIONID']==int(idee)][0]
        
        # append lists with lines and colors
        for c in subway_df['COLOR'].iloc[stat_index]:
            color.append(c)
            
    return color

        
def get_mta_lines(ID,subway_df):
    
    lines=list() #initiate colors
    
    for idee in ID:
        
        #find index of subway station
        stat_index=subway_df.index[subway_df['STATIONID']==int(idee)][0]
        
        # append lists with lines and colors
        for L in subway_df['LINES'].iloc[stat_index]:
            lines.append(L)
            
    return lines        

In [133]:
trimmed_mta_ID_only['COLOR']=trimmed_mta_ID_only['MTA'].map(lambda x: get_mta_color(x,subway_stations_lines_colors))
trimmed_mta_ID_only['LINES']=trimmed_mta_ID_only['MTA'].map(lambda x: get_mta_lines(x,subway_stations_lines_colors))

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [136]:
trimmed_mta_ID_only.head(5)

,JOB_TICKET_OR_WORK_ORDER_ID,BORO_CODE,BOROUGH,ZIP_CODE,LATITUDE,LONGITUDE,INSPECTION_DATE,APPROVED_DATE,LOCATION,n_prev_visits,new_visit,MTA,COLOR,LINES
0,20090,3,Brooklyn,11201,40.692184,-73.985753,2010-05-14 13:46:39,2010-05-18 13:00:01,"(40.6921838341671, -73.9857525784075)",2,True,[377],[yellow],"[N, R]"
14,28921,3,Brooklyn,11201,40.691739,-73.986283,2010-05-14 12:15:33,2010-05-18 13:00:33,"(40.6917392459776, -73.9862827561782)",2,True,[377],[yellow],"[N, R]"
31,31318,3,Brooklyn,11225,40.669580,-73.951250,2010-01-27 08:41:55,2010-01-27 11:02:25,"(40.6695801910464, -73.9512497520348)",0,False,[445],"[red, green]","[3, 4]"
32,31320,3,Brooklyn,11225,40.669443,-73.951189,2010-01-27 09:08:07,2010-01-27 11:03:32,"(40.6694429266035, -73.9511885678396)",0,False,[445],"[red, green]","[3, 4]"
36,32176,1,Manhattan,10029,40.795130,-73.944859,2010-01-20 17:19:09,2010-01-22 13:22:12,"(40.7951302785112, -73.9448588661273)",0,False,[450],[green],"[4, 6, 6 Express]"


In [137]:
trimmed_mta_ID_only.to_csv('data/trimmed_mta_ID_only.csv')

In [140]:
trimmed_mta_ID_only['YEAR_INSPEC']=trimmed_mta_ID_only['INSPECTION_DATE'].map(lambda x:parser.parse(x).year)
trimmed_mta_ID_only['MONTH_INSPEC']=trimmed_mta_ID_only['INSPECTION_DATE'].map(lambda x:parser.parse(x).month)

/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ChrisD/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [141]:
trimmed_mta_ID_only.shape[0]

10123

### Final separate out stations for Groupby

In [142]:
station_trimmed=trimmed_mta_ID_only

In [147]:
s = station_trimmed.apply(lambda x: pd.Series(x['MTA']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'MTA_sep'

station_trimmed=station_trimmed.drop('MTA', axis=1).join(s)

In [148]:
station_group=station_trimmed

In [149]:
#dummy=station_group.groupby(['MTA_sep','YEAR_INSPEC','MONTH_INSPEC']).count()
dummy=station_group.groupby(['MTA_sep','YEAR_INSPEC','MONTH_INSPEC']).count()

In [151]:
dummy.head(2)

JOB_TICKET_OR_WORK_ORDER_ID  BORO_CODE  \
MTA_sep YEAR_INSPEC MONTH_INSPEC                                           
1       2010        5                                       1          1   
        2011        2                                       1          1   

                                  BOROUGH  ZIP_CODE  LATITUDE  LONGITUDE  \
MTA_sep YEAR_INSPEC MONTH_INSPEC                                           
1       2010        5                   1         1         1          1   
        2011        2                   1         1         1          1   

                                  INSPECTION_DATE  APPROVED_DATE  LOCATION  \
MTA_sep YEAR_INSPEC MONTH_INSPEC                                             
1       2010        5                           1              1         1   
        2011        2                           1              1         1   

                                  n_prev_visits  new_visit  COLOR  LINES  
MTA_sep YEAR_INSPEC MONTH_INSPEC                                          
1       2010        5                         1          1      1      1  
        2011        2                         1          1      1      1

In [226]:
# by year and month, grouped by station
grouped_df = station_group.groupby(['MTA_sep','YEAR_INSPEC','MONTH_INSPEC'])
mta_year_month=pd.DataFrame(grouped_df.size().reset_index(name = "Group_Count"))
#mta_year_month['COLOR']='-'
#mta_year_month['LINES']='-'
#get color
mta_year_month['COLOR']=mta_year_month['MTA_sep'].map(lambda x: get_mta_color([x],subway_stations_lines_colors))
#get lines
mta_year_month['LINES']=mta_year_month['MTA_sep'].map(lambda x: get_mta_lines([x],subway_stations_lines_colors))

# by year, grouped by station
grouped_df = station_group.groupby(['MTA_sep','YEAR_INSPEC'])
mta_year=pd.DataFrame(grouped_df.size().reset_index(name = "Group_Count"))
#mta_year['COLOR']='-'
#mta_year['LINES']='-'
#get color
mta_year['COLOR']=mta_year['MTA_sep'].map(lambda x: get_mta_color([x],subway_stations_lines_colors))
#get lines
mta_year['LINES']=mta_year['MTA_sep'].map(lambda x: get_mta_lines([x],subway_stations_lines_colors))

In [227]:
mta_year.head(10)
#mta_year.shape[0]

,MTA_sep,YEAR_INSPEC,Group_Count,COLOR,LINES
0,1,2010,1,[green],"[4, 6, 6 Express]"
1,1,2011,2,[green],"[4, 6, 6 Express]"
2,1,2015,1,[green],"[4, 6, 6 Express]"
3,1,2016,3,[green],"[4, 6, 6 Express]"
4,100,2016,1,[yellow],[Q]
5,100,2017,1,[yellow],[Q]
6,101,2011,1,[purple],"[7, 7 Express]"
7,101,2014,4,[purple],"[7, 7 Express]"
8,101,2015,2,[purple],"[7, 7 Express]"
9,101,2017,1,[purple],"[7, 7 Express]"


In [228]:
print('MTA_sep type in mta_year:',type(mta_year['MTA_sep'].iloc[10]))
print('STATIONID type in subway_stations_lines_colors:',type(subway_stations_lines_colors['STATIONID'].iloc[10]))

MTA_sep type in mta_year: <class 'str'>
STATIONID type in subway_stations_lines_colors: <class 'numpy.int64'>


In [229]:
mta_year['MTA_sep']=pd.to_numeric(mta_year['MTA_sep'])
mta_year_month['MTA_sep']=pd.to_numeric(mta_year_month['MTA_sep'])

In [230]:
#change MTA_sep ID to numeric for merge
mta_year['MTA_sep']=pd.to_numeric(mta_year['MTA_sep'])
mta_year_month['MTA_sep']=pd.to_numeric(mta_year_month['MTA_sep'])

#merge with 
mta_year=mta_year.merge(subway_stations_lines_colors, left_on='MTA_sep', right_on='STATIONID', how='inner')
mta_year_month=mta_year_month.merge(subway_stations_lines_colors, left_on='MTA_sep', right_on='STATIONID', how='inner')

#trim mta_year
mta_year=mta_year[['MTA_sep','NAME','LINE','YEAR_INSPEC','Group_Count','GPS','ZIP','LINES_y','COLOR_y']]
mta_year.columns=['MTA_ID', 'NAME', 'ROUTE', 'YEAR_INSPEC','COUNT', 'GPS', 'ZIP', 'LINES',
       'COLOR']

#trim mta_year_month
mta_year_month=mta_year_month[['MTA_sep','NAME','LINE','YEAR_INSPEC','MONTH_INSPEC','Group_Count','GPS','ZIP','LINES_y','COLOR_y']]
mta_year_month.columns=['MTA_ID', 'NAME', 'ROUTE', 'YEAR_INSPEC', 'MONTH_INSPEC','COUNT', 'GPS', 'ZIP',
       'LINES', 'COLOR']


In [231]:
mta_year.head(2)

,MTA_ID,NAME,ROUTE,YEAR_INSPEC,COUNT,GPS,ZIP,LINES,COLOR
0,1,Astor Pl,4-6-6 Express,2010,1,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
1,1,Astor Pl,4-6-6 Express,2011,2,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]


##### create unique station name

In [232]:
mta_year['STATION']=mta_year['NAME'] + ' : ' + mta_year['ROUTE']
mta_year_month['STATION']=mta_year['NAME'] + ' : ' + mta_year['ROUTE']

mta_year=mta_year[['MTA_ID', 'STATION','NAME', 'ROUTE', 'YEAR_INSPEC','COUNT', 'GPS', 'ZIP',
       'LINES', 'COLOR']]
mta_year_month=mta_year_month[['MTA_ID', 'STATION','NAME', 'ROUTE', 'YEAR_INSPEC', 'MONTH_INSPEC','COUNT', 'GPS', 'ZIP',
       'LINES', 'COLOR']]

In [233]:
mta_year.head(2)

,MTA_ID,STATION,NAME,ROUTE,YEAR_INSPEC,COUNT,GPS,ZIP,LINES,COLOR
0,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2010,1,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
1,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2011,2,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]


In [234]:
mta_year.head(20)

,MTA_ID,STATION,NAME,ROUTE,YEAR_INSPEC,COUNT,GPS,ZIP,LINES,COLOR
0,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2010,1,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
1,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2011,2,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
2,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2015,1,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
3,1,Astor Pl : 4-6-6 Express,Astor Pl,4-6-6 Express,2016,3,"(-73.99106999861966, 40.73005400028978)",10003,"[4, 6, 6 Express]",[green]
4,100,Ocean Pkwy : Q,Ocean Pkwy,Q,2016,1,"(-73.96850099975177, 40.57631166708091)",11235,[Q],[yellow]
5,100,Ocean Pkwy : Q,Ocean Pkwy,Q,2017,1,"(-73.96850099975177, 40.57631166708091)",11235,[Q],[yellow]
6,101,Vernon Blvd - Jackson Ave : 7-7 Express,Vernon Blvd - Jackson Ave,7-7 Express,2011,1,"(-73.95358099875249, 40.74262599969749)",11101,"[7, 7 Express]",[purple]
7,101,Vernon Blvd - Jackson Ave : 7-7 Express,Vernon Blvd - Jackson Ave,7-7 Express,2014,4,"(-73.95358099875249, 40.74262599969749)",11101,"[7, 7 Express]",[purple]
8,101,Vernon Blvd - Jackson Ave : 7-7 Express,Vernon Blvd - Jackson Ave,7-7 Express,2015,2,"(-73.95358099875249, 40.74262599969749)",11101,"[7, 7 Express]",[purple]
9,101,Vernon Blvd - Jackson Ave : 7-7 Express,Vernon Blvd - Jackson Ave,7-7 Express,2017,1,"(-73.95358099875249, 40.74262599969749)",11101,"[7, 7 Express]",[purple]


In [235]:
mta_year.to_csv('data/mta_100m_rad_year.csv')
mta_year_month.to_csv('data/mta_100m_rad_year_month.csv')